# Computación Científica - Tarea 3
---
Vicente Lizana Estivill  
vlizana@alumnos.inf.utfsm.cl  
201310004-K

---

## Introducción
---



En esta tarea analizaremos los diferentes métodos de resolución de _Ordinary Differential Equations_, específicamente _Initial Value Problems_. Veremos como se comportan a medida que vamos variando el tamaño de los saltos que hacemos en el cálculo iterativo y estudiaremos el comportamiento de sus órbitas.

---

## Desarrollo y Análisis de Resultados
---

### Bibliotecas y Utilidades
---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, fixed, IntSlider

In [2]:
def F_test(t,y):
    y1 = y[0]
    y2 = y[1]
    return np.array([y1*y2 + y2**2, y2**2 - y1**3])

In [3]:
def Euler_Method(y,t,f,h):
    return y+h*f(t,y)

def Trapezoidal_Rule(y,t,f,h):
    k1=y+h/2.0*Euler_Method(y,t,f,h)
    return y+h*f(t+h/2.0,k1)

def RK4(y,t,f,h):
    k1=f(t,y)
    k2=f(t+h/2.0,y+(h/2.0)*k1)
    k3=f(t+h/2.0,y+(h/2.0)*k2)
    k4=f(t+h,y+h*k3)
    return y+(h/6.0)*(k1+2.0*k2+2.0*k3+k4)

In [4]:
def plot_y_t_2D(time, y_output, i=1):
    fig = plt.figure(figsize=(17,6))
    
    gs = gridspec.GridSpec(1, 2, width_ratios=[2, 2])
    #First Subplot
    ax0 = plt.subplot(gs[0])
    ax0.grid(True)
    plt.xlabel(r"$t$",fontsize = 20) 
    plt.plot(time[:i],y_output[0,:i],'b-',label=r"$y_1(t)$")
    plt.plot(time[:i],y_output[1,:i],'y-',label=r"$y_2(t)$")
    plt.legend(loc='best', borderaxespad=0., fontsize = 20)
    ax0.axis([0, np.pi, -2, 2])
    
    ax1 = plt.subplot(gs[1])
    #Second Subplot
    ax1.plot(y_output[0,:i],y_output[1,:i],'g-')
    ax1.axis([-2, 2, -2, 2])
    ax1.set_xlabel(r"$y_1$",fontsize=20)
    ax1.set_ylabel(r'$y_2$',fontsize=20)
    plt.grid()
    plt.show()

In [5]:
elev_widget = IntSlider(min=0, max=180, step=10, value=20)
azim_widget = IntSlider(min=0, max=360, step=10, value=150)

---

### Pregunta 1
---

Se tiene el siguiente problema de valor inicial no-autonomo definido para $t \in [0,T]$:
$$
\begin{align}
y_1'(t) &= \displaystyle \frac{-y_1(t)y_2(t)}{20} + y_2^2(t)\sin(t) \\
y_2'(t) &= \displaystyle \frac{y_1^2(t)}{20} - y_1(t)y_2(t)\sin(t) \\
y_1(0) &= \cos(\theta) \\
y_2(0) &= \sin(\theta)
\end{align}
$$

Con $\theta = \displaystyle \frac{4\pi}{5}.$

---

**a.-** Demuestre que las órbitas del sistema dinámico se mueven en un círculo y determine el radio del círculo.

---

Sea $\gamma_1$ la órbita tal que $\gamma_1 = \langle y_1(t), y_2(t) \rangle$, con $t \in [0, \pi]$. Podemos ver que cada punto de esta órbita está definido en función de $t$. Como la función $\sin(x)$ es biyectiva en el intervalo $[0, \pi]$, entonces $\sin(t_i) = \sin(t_j) \Rightarrow t_i = t_j$. Bajo este principio tenemos que:

$$ sin(t) = \frac{y_1'}{y_2^2} + \frac{y_1}{20 \cdot y_2} = -\frac{y_2'}{y_1 \cdot y_2} + \frac{y_1}{20 \cdot y_2} $$

$$ \frac{y_1'}{y_2} = -\frac{y_2'}{y_1} $$

$$ y_1\ y_1' + y_2\ y_2' = 0 $$

$$ \int y_1\ \frac{dy_1}{dt} dt + \int y_2\ \frac{dy_2}{dt} dt = 0 $$

$$ \int y_1\ dy_1 + \int y_2\ dy_2 = 0 $$

$$ \frac{y_1^2}{2} + \frac{y_2^2}{2} = \hat{c} $$

$$ y_1^2 + y_2^2 = c = r^2 $$

Que es precisamente la ecuación de una circunferencia.

Para determinar el radio utilizamos los valores que tenemos disponibles, $y_1(0)$ y $y_2(0)$, ya que el radio es constante.

$$ y_1^2(0) + y_2^2(0) = r^2 $$

$$ \sin^2(\theta) + \cos^2(\theta) = r^2 $$

$$ r = 1 $$

---

**b.-** Modifique la función **plot_y_t_2D** para las curvas $\langle y_i, t \rangle$ con $i \in \{1,2\}$ para $T \in [0,\pi]$ del enunciado. Se debe crear un *widget* hasta el tiempo $t_i$ y $h$, mostrando la solución de $y_i$ con los 3 *solvers*, además de la curva paramétrica $\langle y_1, y_2 \rangle$ para los 3 *solvers*. Para resolver las IVP's debe seguir la siguiente estructura:

```python
'''
solver    - (string) Type of solver to use
F         - (function) Function of the dynamic system F(t,y)
x0        - (1D-array) Initial guess
time_grid - (float) times where solution will be computed
x_sol     - (nD-array) Array of solutions (could be 2D or 3D).
'''
```

```python     
def IVP_Method(solver, F, x0, time_grid):
    #...
    return x_sol
```

---

In [6]:
def IVP_Method(solver, F, x0, time_grid, N=1):
    h = time_grid/N
    t_times = np.arange(0, time_grid+h, h)
    x_sol = np.zeros([x0.size, t_times.size])
    x_sol[:,0] = x0
    for i in range(1, t_times.size):
        x_sol[:,i] = solver(x_sol[:,i-1],t_times[i-1],F,h)
    return x_sol

In [7]:
def modified_plot_y_t_2D(solver, F, x0, T, N=1):
    h = T/N
    time = np.arange(0, T+h, h)
    y_output = IVP_Method(solver, F, x0, T, N)
    
    fig = plt.figure(figsize=(17,6))
    gs = gridspec.GridSpec(1, 2, width_ratios=[2, 2])
    
    #First Subplot
    ax0 = plt.subplot(gs[0])
    ax0.grid(True)
    plt.xlabel(r"$t$",fontsize = 20) 
    plt.plot(time,y_output[0,:],'b-',label=r"$y_1(t)$")
    plt.plot(time,y_output[1,:],'y-',label=r"$y_2(t)$")
    plt.legend(loc='best')
    ax0.axis([0, np.pi, -2, 2])
    
    ax1 = plt.subplot(gs[1])
    ######
    u = np.linspace(0, 2*np.pi, 100)
    x_c = np.sin(u)
    y_c = np.cos(u)
    ax1.plot(x_c, y_c, 'y-', label='reference')
    ######
    #Second Subplot
    ax1.plot(y_output[0,:],y_output[1,:],'g-', label='curve')
    ax1.axis([-2, 2, -2, 2])
    ax1.set_xlabel(r"$y_1$",fontsize=20)
    ax1.set_ylabel(r'$y_2$',fontsize=20)
    plt.legend()
    plt.grid()
    plt.show()

In [8]:
def F1(t, y):
    y1 = y[0]
    y2 = y[1]
    return np.array([
        -y1*y2/20 + y2**2*np.sin(t),
        y1**2/20 - y1*y2*np.sin(t)
    ])

y0_1 = np.array([
    np.cos(4*np.pi/5),
    np.sin(4*np.pi/5)
])

In [9]:
interact(
    modified_plot_y_t_2D,
    solver = {
        "Euler" : Euler_Method,
        "Trapezoid" : Trapezoidal_Rule,
        "RK4" : RK4
    },
    F = fixed(F1),
    x0 = fixed(y0_1),
    T = fixed(np.pi),
    N = (1, 1000)
)

A Jupyter Widget

<function __main__.modified_plot_y_t_2D>

* Podemos ver que con Euler tenemos una relativamente buena aproximación a partir de las $50$ iteraciones aproximadamente, esto nos indica un $h = \frac{\pi}{50} \approx 0.063$.
* Lo mismo pasa con Trapezoide.
* RK4 aproxima mejor con $6$ iteraciones que los demás con $50$. $h = \frac{\pi}{6} \approx 0.524$.

---

### Pregunta 2
---

Se tiene el siguiente problema de valor inicial no-autonomo definito para $t \in [0,T]$:
$$
\begin{array}{lll}
y_1'(t) &= \displaystyle \frac{y_2^2(t)}{5} - y_3^2(t)y_2(t)\sin(t) &  \quad \quad &(1)
\\
y_2'(t) &= \displaystyle y_1^2(t)y_3(t) - \frac{y_1(t)y_2(t)}{5} & \quad \quad &(2)
\\
y_3'(t) &= \displaystyle y_1(t)y_2(t)y_3(t)\sin(t) - y_{1}^2(t) y_2(t) & \quad \quad &(3)
\\
\\
y_1(0) &= \sin(\theta) & \quad \quad &(4)
\\
y_2(0) &= \cos(\theta) & \quad \quad &(5)
\\
y_3(0) &= 0 & \quad \quad &(6)
\end{array}
$$

Con $\theta = \displaystyle \frac{5\pi}{4}.$

---

**a.-** Demuestre que las órbitas del sistema dinámico se mueven en una esfera y determine el radio de la esfera.

---

Usando el mismo razonamiento que en la pregunta anterior:

$$ \sin(t) = \frac{y_2^2 - 5y_1'}{5\ y_3^2\ y_2} = \frac{y_3' + y_1^2\ y_2}{y_1\ y_2\ y_3} $$

$$ y_2^2\ y_1 - 5y_1'\ y_1 = 5\ y_3'\ y_3 + 5\ y_1^2\ y_2\ y3 $$

$$ y_2^2\ y_1 - 5\ y_1^2\ y_2\ y3 = 5\ y_3'\ y_3 + 5y_1'\ y_1  $$

$$ -5\ y_2 \left(y_1^2\ y3 - \frac{y_2\ y_1}{5}\right) = 5\ y_3'\ y_3 + 5\ y_1'\ y_1  $$

$$ 0 = y_1'\ y_1 + y_2'\ y_2 + y_3'\ y_3 $$

$$ \int y_1 \frac{dy_1}{dt} dt + \int y_2 \frac{dy_2}{dt} dt + \int y_3 \frac{dy_3}{dt} dt = 0 $$

$$ \int y_1 dy_1 + \int y_2 dy_2 + \int y_3 dy_3 = 0 $$

$$ \frac{y_1^2}{2} + \frac{y_2^2}{2} + \frac{y_3^2}{2} = \hat{c} $$

$$ y_1^2 + y_2^2 + y_3^2 = c $$

Que es precisamente la ecuación de una esfera.

Nuevamente utilizamos los valores disponibles para sacar el radio constante.

$$ y_1^2(0) + y_2^2(0) + y_3^2(0) = r^2 $$

$$ r^2 = \sin^2(\theta) + \cos^2(\theta) + 0^2 $$

$$ r = 1 $$

---

**b.-** Cree la función **plot_y_t_3D**, e implemente un *widget* hasta el tiempo $t_i$ y $h$ en los dos *subplots*:

* $\langle y^{solver}_i, t \rangle$ para $i \in \{1,2,3\}$ en 2D que muestre el avance de la curva $y_i$ para los 3 *solvers* hasta $t_i$.

* $\langle y_1(t), y_2(t), y_3(t) \rangle$ en 3D donde se muestre la trayectoria de una partícula hasta $t_i$ para los 3 *solvers*, graficando además una esfera con el radio encontrado en la pregunta 2.a) como referencia para los puntos graficados. Use los *elev_widget* y *azim_widget* para poder girar su gráfico y verificar que efectivamente los puntos se encuentran en la esfera.

---

In [10]:
def plot_y_t_3D(solver, F, x0, T, elev, azim, N=1):
    h = T/N
    time = np.arange(0, T+h, h)
    y_output = IVP_Method(solver, F, x0, T, N)

    fig = plt.figure(figsize=(17,6))
    gs = gridspec.GridSpec(1, 2, width_ratios=[2, 2])

    #First Subplot
    ax0 = plt.subplot(gs[0])
    ax0.grid(True)
    
    plt.xlabel(r"$t$",fontsize = 20) 
    plt.plot(time,y_output[0,:],'b-',label=r"$y_1(t)$")
    plt.plot(time,y_output[1,:],'y-',label=r"$y_2(t)$")
    plt.plot(time,y_output[2,:],'r-',label=r"$y_3(t)$")
    plt.legend(loc='best')
    ax0.axis([0, np.pi, -2, 2])
    
    ax1 = plt.subplot(gs[1], projection='3d')
    ax1 = fig.gca()
    ax1.view_init(elev,azim)
    ########
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)

    x = np.outer(np.cos(u), np.sin(v))
    y = np.outer(np.sin(u), np.sin(v))
    z = np.outer(np.ones_like(u), np.cos(v))
    
    ax1.plot_surface(x, y, z, cmap=plt.cm.winter)
    ########
    #Second Subplot
    ax1.plot(y_output[0,:],y_output[1,:],y_output[2,:],'r-')
    ax1.set_xlabel(r"$y_1$",fontsize=20)
    ax1.set_ylabel(r'$y_2$',fontsize=20)
    ax1.set_zlabel(r'$y_3$',fontsize=20)
    plt.show()

In [11]:
def F2(t, y):
    y1 = y[0]
    y2 = y[1]
    y3 = y[2]
    return np.array([
        y2**2/5 + y3**2*y2*np.sin(t),
        y1**2*y3 - y1*y2/5,
        y1*y2*y3*np.sin(t) - y1**2*y2
    ])

y0_2 = np.array([
    np.cos(4*np.pi/5),
    np.sin(4*np.pi/5),
    0
])

In [12]:
interact(
    plot_y_t_3D,
    solver = {
        "Euler" : Euler_Method,
        "Trapezoid" : Trapezoidal_Rule,
        "RK4" : RK4
    },
    F = fixed(F2),
    x0 = fixed(y0_2),
    T = fixed(np.pi),
    elev=elev_widget,
    azim=azim_widget,
    N = (1, 1000)
)

A Jupyter Widget

<function __main__.plot_y_t_3D>

Esta no es una curva que nos aporte mucha información ya que no parece presentar cambios significativos más allá de las 15 iteraciones en los dos primeros métodos y RK4 se aproxima incluso con un par de iteraciones, ya con $4$ se ve relativamente saturado el método.

---

### Pregunta 3
---

**a.-** Construya un problema de valor inicial no-autonomo definido para $t \in [0,T]$ que viva en la elipsoide $\displaystyle \Big(\frac{x}{2}\Big)^2 + \Big(\frac{y}{0.8}\Big)^2 + \Big(\frac{z}{1.5}\Big)^2=1$ con sus respectivas condiciones iniciales.

---

Realizamos el proceso inverso que en los ejercicios anteriores, partir de la ecuación, derivar y llegar a la forma de _IVP_:

$$ \frac{x^2}{4} + \frac{25y^2}{16} + \frac{4z^2}{9} = 1 $$

$$ 36\ x^2(t) + 225\ y^2(t) + 64\ z^2(t) = 144\quad \Big/\ \frac{d}{dt} $$

$$ 36\ x(t)\ x'(t) + 225\ y(t)\ y'(t) + 64\ z(t)\ z'(t) = 0 $$

Acá podemos invevntar dos de las derivadas con tal de que la última cumpla con la ecuación. Como el sistema debe ser no autónomo, propongo:

$$ x'(t) = t \cdot y(t) $$

$$ z'(t) = t \cdot \frac{y(t)\ x(t)}{z(t)} $$

Luego

$$ y'(t) = -\frac{64\ z(t)\ z'(t) + 36\ x(t)\ x'(t)}{225\ y(t)}
= -\frac{64}{225}\ t \cdot x(t) -\frac{36}{255}\ t \cdot x(t) = -\frac{4}{9}\ t \cdot x(t) $$

Por último solo basta con elegir de manera conveniente el valor inicial, de tal forma que cumpla también con la ecuación de la elipsoide.

$$ x(0) = 2 \cdot \sin(\theta) $$

$$ z(0) = 1.5 \cdot \cos(\theta) $$

$$ y(0) = 0 $$

con $\theta = \frac{4}{5}\pi $.

---

**b.-** Modifique la función **plot_y_t_3D** para el problema 3.a), colocando como referencia la elipsoide propuesta.

---

In [13]:
def modified_plot_y_t_3D(solver, F, x0, T, elev, azim, N=1):
    h = T/N
    time = np.arange(0, T+h, h)
    y_output = IVP_Method(solver, F, x0, T, N)
    
    fig = plt.figure(figsize=(17,6))
    gs = gridspec.GridSpec(1, 2, width_ratios=[2, 2])

    #First Subplot
    ax0 = plt.subplot(gs[0])
    ax0.grid(True)
    
    plt.xlabel(r"$t$",fontsize = 20) 
    plt.plot(time,y_output[0,:],'b-',label=r"$x(t)$")
    plt.plot(time,y_output[1,:],'y-',label=r"$y(t)$")
    plt.plot(time,y_output[2,:],'r-',label=r"$z(t)$")
    plt.legend(loc='best')
    ax0.axis([0, np.pi, -2, 2])
    
    ax1 = plt.subplot(gs[1], projection='3d')
    ax1 = fig.gca()
    ax1.view_init(elev,azim)
    ########
    rx = 2.0
    ry = 0.8
    rz = 1.5

    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)

    x = rx * np.outer(np.cos(u), np.sin(v))
    y = ry * np.outer(np.sin(u), np.sin(v))
    z = rz * np.outer(np.ones_like(u), np.cos(v))

    ax1.plot_surface(x, y, z, cmap=plt.cm.winter)
    ########
    #Second Subplot
    ax1.plot(y_output[0,:],y_output[1,:],y_output[2,:],'r-')
    ax1.set_xlabel(r"$x$",fontsize=20)
    ax1.set_ylabel(r'$y$',fontsize=20)
    ax1.set_zlabel(r'$z$',fontsize=20)
    plt.show()

In [14]:
def F3(t, v):
    x = v[0]
    y = v[1]
    z = v[2]
    return np.array([
        t * y,
        -4 * t * x / 9.0,
        t * y * x / z
    ])

y0_3 = np.array([
    2 * np.sin(4*np.pi/5),
    0,
    1.5 * np.cos(4*np.pi/5)
])

In [15]:
interact(
    modified_plot_y_t_3D,
    solver = {
        "Euler" : Euler_Method,
        "Trapezoid" : Trapezoidal_Rule,
        "RK4" : RK4
    },
    F = fixed(F3),
    x0 = fixed(y0_3),
    T = fixed(np.pi),
    elev=elev_widget,
    azim=azim_widget,
    N = (1, 1000)
)

A Jupyter Widget

<function __main__.modified_plot_y_t_3D>

Acá se logra apreciar mejor el error de arrastre.
* Con 100 iteraciones ni Euler ni Trapezoide se aprecian sobre la superficie del elipsoide. Con 150 ya se aprecia una mejor aproximación, $h = \frac{\pi}{150} \approx 0.021$.
* Debido a lo larga y variable que es la curva (se requieren más segmentos para suavizarla), RK4 hace un buen trabajo aproximadamente desde las $20$ iteraciones, $h = \frac{\pi}{20} \approx 0.157$.

---

## Conclusiones
---

* Se puede ver que RK4 es mucho mejor que los demás, ya que disminuye casi completamente el error de arrastre que conlleva este problema, es decir, a pesar de que cada punto se calcula a partir del anterior, los puntos en RK4 siempre quedan muy cerca de donde deberían estar.
* Euler y Trapezoide se comportan de la misma manera, esto se explica por que tienen el mismo orden de convergencia cuadrático con respecto al valor de $h$.

---

## Referencias
---

[Notebook de IVP](https://www.dropbox.com/sh/3eef0gtr3tjnech/AABBOs7zvkVZYhoje8GqaToYa/Jupyter%20notebooks/CC2?dl=0&preview=U3_AdvancedApplications.ipynb)    
[Documentación de Matplotlib](https://matplotlib.org/api/pyplot_api.html)    
[Documentación de SciPy](https://docs.scipy.org/doc/scipy/reference/)

---